In [56]:
import numpy as np
import pandas as pd
import os

In [57]:
def get_label(sample: str) -> str:
    """Gets label of a given sample

    Args:
        sample (str): a filename

    Returns:
        str: label (M or K)
    """
    return sample.split('.')[0][-1]


samples = os.listdir('./data/')
labels = [get_label(sample) for sample in samples]
train = pd.DataFrame({'File' : samples, 'Label' : labels})

In [58]:
import scipy.io.wavfile
def HPS(filename: str) -> str:
    male_freqs=[60,160]
    female_freqs=[180,270]
    
    w, signal = scipy.io.wavfile.read(f'data/{filename}')

    sample_length = len(signal)/w  # length in seconds

    parts = [signal[i*w:(i+1)*w] for i in range(int(sample_length))]  # each part lasts exactly one second

    results = []

    for part in parts:
        hamming_window = np.hamming(len(part))
        data = part*hamming_window
        abs_fft = np.abs(np.fft.fft(data))/w
        fft_r = np.copy(abs_fft)
        for i in range(2, 5):
            tab = np.copy(abs_fft[::i])
            fft_r = fft_r[:len(tab)]
            fft_r *= tab
        results.append(fft_r)

    result = [0]*len(results[int(len(results)/2)])

    for res in results:
        result += res

    if(sum(result[male_freqs[0]:male_freqs[1]]) > sum(result[female_freqs[0]:female_freqs[1]])): return 'M'
    else: return 'K'

In [59]:
y_pred = [HPS(file) for file in samples]
train['Pred'] = y_pred

In [60]:
from sklearn.metrics import accuracy_score

accuracy_score(train['Label'], train['Pred'])

0.945054945054945